# Project 2


### This section takes the dataframe from part 1 and cleans it for modeling.

In [50]:
import pandas as pd
import numpy as np

In [43]:
# Save as pickle
import pickle as pkl
with open("df900.pkl", 'rb') as picklefile: 
    df900 = pkl.load(picklefile)
df900[0:6]

,Company,Reviews,CEO_Reviews,Founded,Headquarters,Private_Public,Benefit_Rating,Revenue,Industry,Size
0,Waggener Edstrom,3.8\nRating Trends,93%,1983,"Headquarters Bellevue, WA",Company - Private,3.8\n32 Ratings,Unknown / Non-Applicable,Advertising & Marketing,501 to 1000 employees
1,InVisionApp,4.7\nRating Trends,90%,2011,"Headquarters New York, NY",Company - Private,4.2\n17 Ratings,Unknown / Non-Applicable,Internet,501 to 1000 employees
2,New Relic,4.2\nRating Trends,95%,2008,"Headquarters San Francisco, CA",Company - Public (NEWR),4.0\n46 Ratings,$100 to $500 million (USD) per year,Computer Hardware & Software,1001 to 5000 employees
3,Akraya,3.5\nRating Trends,84%,2001,"Headquarters Santa Clara, CA",Company - Private,3.5\n4 Ratings,$25 to $50 million (USD) per year,Staffing & Outsourcing,201 to 500 employees
4,Shyft Technologies,5.0\nRating Trends,100%,2016,"Headquarters Seattle, WA",Company - Private,5.0\n1 Rating,Unknown / Non-Applicable,Computer Hardware & Software,1 to 50 employees
5,Henkel,3.4\nRating Trends,74%,1876,Headquarters Düsseldorf (Germany),Company - Public (HEN),3.9\n74 Ratings,$10+ billion (USD) per year,Consumer Products Manufacturing,10000+ employees


In [12]:
df900.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 10 columns):
Company           286 non-null object
Reviews           286 non-null object
CEO_Reviews       286 non-null object
Founded           286 non-null object
Headquarters      286 non-null object
Private_Public    286 non-null object
Benefit_Rating    286 non-null object
Revenue           286 non-null object
Industry          286 non-null object
Size              286 non-null object
dtypes: object(10)
memory usage: 22.4+ KB


In [44]:
''' This changes grabs the relevant parts of each string, changes the string values to floating point,
and changes unknown values to "NaN".
'''
df900['Reviews'] = (df900['Reviews'].str.replace('N/A', 'NaN'))
df900['Reviews_clean'] = (df900['Reviews'].astype(str).str[0:3]).astype(float)
df900['Benefit_Rating'] = (df900['Benefit_Rating'].str.replace('NA', 'NaN'))
df900['Benefit_Rating_clean'] = (df900['Benefit_Rating'].astype(str).str[0:3]).astype(float)
df900['CEO_Reviews'] = df900['CEO_Reviews'].str.replace('N/A', 'NaN')
df900['CEO_Reviews'] = df900['CEO_Reviews'].str.replace('Na', 'NaN')
df900['CEO_Reviews_clean'] = df900['CEO_Reviews'].str[:-1].astype(float)
df900['Founded_clean'] = df900['Founded'].str.replace('Unknown', 'NaN')
df900['Revenue_clean'] = df900['Revenue'].str.replace('Unknown / Non-Applicable', 'NaN' )
df900['Revenue_clean2'] = df900['Revenue_clean'].str.replace('NaN per year', 'NaN')
df900.shape

(286, 16)

In [47]:
#Relabeling revenue featuree
conditions = [
    (df900['Revenue_clean2'] == 'Less than $1 million (USD) per year'), 
    (df900['Revenue_clean2'] == '$1 to $5 million (USD) per year'),
    (df900['Revenue_clean2'] == '$5 to $10 million (USD) per year'),
    (df900['Revenue_clean2'] == '$10 to $25 million (USD) per year'),
    (df900['Revenue_clean2'] == '$25 to $50 million (USD) per year'),
    (df900['Revenue_clean2'] == '$50 to $100 million (USD) per year'),
    (df900['Revenue_clean2'] == '$100 to $500 million (USD) per year'),
    (df900['Revenue_clean2'] == '$500 million to $1 billion (USD) per year'),
    (df900['Revenue_clean2'] == '$1 to $2 billion (USD) per year'),
    (df900['Revenue_clean2'] == '$2 to $5 billion (USD) per year'),
    (df900['Revenue_clean2'] == '$5 to $10 billion (USD) per year'),
    (df900['Revenue_clean2'] == '$10+ billion (USD) per year'),
    (df900['Revenue_clean2'] == 'NaN')]
choices = [int(1), int(5),int(10), int(25), int(50), int(100), int(500),int (1000), int(2000),
           int(5000), int(10000),int(50000),'NaN']
df900['Revenue_clean2'] = np.select(conditions, choices)

In [48]:
#Relabeling size
conditions = [
    (df900['Size'] == '1 to 50 employees'), 
    (df900['Size'] == '51 to 200 employees'),
    (df900['Size'] == '201 to 500 employees'),
    (df900['Size'] == '501 to 1000 employees'),
    (df900['Size'] == '1001 to 5000 employees'),
    (df900['Size'] == '5001 to 10000 employees'),
    (df900['Size'] == '10000+ employees'),
    (df900['Size'] == 'Unknown') ]
choices = [int(1), int(51),int(201), int(501), int(1001), int(5001),int (10001), 'NaN']
df900['Size_clean'] = np.select(conditions, choices)

In [51]:
#Greater Seattle Headquarters are labeled "1" Everything else is "0"
df900['Headquarters'].iloc[[5]] = 0
conditions = [
    (df900['Headquarters'] == 'Headquarters Seattle, WA') | 
    (df900['Headquarters'] == 'Headquarters Bellevue, WA') | 
    (df900['Headquarters'] == 'Headquarters Kirkland, WA') |
    (df900['Headquarters'] == 'Headquarters Mountlake Terrace, WA') |
    (df900['Headquarters'] == 'Headquarters Tukwila, WA')|
    (df900['Headquarters'] == 'Headquarters Mercer Island, WA') |
    (df900['Headquarters'] == 'Headquarters Bothell, WA') | 
    (df900['Headquarters'] == 'Headquarters Redmond, WA') |
    (df900['Headquarters'] == 'Headquarters Renton, WA')]

   
choices = [1]
df900['Headquarters_clean'] = np.select(conditions, choices)

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [53]:
# Private is labeled "1".  Everything else is set to "0".
maskpub = df900['Private_Public'].apply(lambda x: 'Public' in x)
conditions = [
    (maskpub),
    (df900['Private_Public'] == 'Company - Private'), 
    (df900['Private_Public'] == 'Nonprofit Organization'),
    (df900['Private_Public'] == 'College / University'), 
    (df900['Private_Public'] == 'Hospital'), 
    (df900['Private_Public'] == 'Private Practice / Firm'),
    (df900['Private_Public'] == 'School / School District'),
    (df900['Private_Public'] == 'Government'),
    (df900['Private_Public'] =='Subsidiary or Business Segment'),
    (df900['Private_Public'] == 'Unknown')]
   
choices = [1,0, 0, 0,0, 0,0,
           0, 0, 'NaN']

df900['Private_public_clean'] = np.select(conditions, choices)

In [54]:
#All NaN need to be changed to floating points so they can be removed
df900['Revenue_clean2'] = pd.to_numeric(df900['Revenue_clean2'],errors = 'coerce')
df900['Founded_clean']= pd.to_numeric(df900['Founded_clean'],errors = 'coerce')
df900['Size_clean']= pd.to_numeric(df900['Size_clean'],errors = 'coerce')
df900['Private_public_clean']= pd.to_numeric(df900['Private_public_clean'],errors = 'coerce')
df900.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 286 entries, 0 to 285
Data columns (total 19 columns):
Company                 286 non-null object
Reviews                 286 non-null object
CEO_Reviews             286 non-null object
Founded                 286 non-null object
Headquarters            286 non-null object
Private_Public          286 non-null object
Benefit_Rating          286 non-null object
Revenue                 286 non-null object
Industry                286 non-null object
Size                    286 non-null object
Reviews_clean           277 non-null float64
Benefit_Rating_clean    250 non-null float64
CEO_Reviews_clean       255 non-null float64
Founded_clean           235 non-null float64
Revenue_clean           286 non-null object
Revenue_clean2          196 non-null float64
Size_clean              285 non-null float64
Headquarters_clean      286 non-null int64
Private_public_clean    284 non-null float64
dtypes: float64(7), int64(1), object(11)
memory usage

In [55]:
#Removing rows with NaN's 
df900.dropna(0,how = 'any', inplace=True)
df900.shape

(145, 19)

In [57]:
'''There are many duplicate companies with different names:  Fred Hutch, Fred Hutchison, etc.
Find duplicates by all columns except company to verify, then sort.  Verify they are indeed duplicates with 
the same values except company.  Then delete the first duplicate value.
'''
#Use original column values (not cleaned versions)

# y = df900[df900.duplicated(subset=['Reviews', 'CEO_Reviews', 'Founded', 'Headquarters','Private_Public', 'Benefit_Rating', 'Revenue', 'Industry', 'Size'], keep=False)]
# y.sort_values(['CEO_Reviews','Founded'])

df900.drop_duplicates(subset=['Reviews', 'CEO_Reviews', 'Founded', 'Headquarters','Private_Public', 'Benefit_Rating', 'Revenue', 'Industry', 'Size'], keep='first', inplace = True)

In [ ]:
df900.to_csv('df900_clean_add.csv')